# TensorFlow Model Garden を使った物体検出

## インポート

In [ ]:
import os
from pathlib import Path

import certifi
import numpy as np
from PIL import Image

import tensorflow as tf
import tensorflow_hub as hub

## 可視化ツール

検出したバウンディングボックスを可視化するために TensorFlow Object Detection API を使用します。
まずはリポジトリをクローンしてください。

In [ ]:
%%bash

if [[ ! -d "models" ]]; then
    git clone --depth 1 https://github.com/tensorflow/models
fi

Object Detection API をインストールします。

In [ ]:
%%bash

cd models/research
protoc object_detection/protos/*.proto --python_out=.
cp -f object_detection/packages/tf2/setup.py .
pip install -q .

後に使用するモジュールをインポートします。

In [ ]:
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.utils import ops as utils_ops

## ラベルマップデータをロード

インデックスから物体のカテゴリ名を対応付けるラベルマップデータをロードします。

In [ ]:
category_index = label_map_util.create_category_index_from_labelmap(
    "models/research/object_detection/data/mscoco_label_map.pbtxt",
    use_display_name=True,
)

## モデルをダウンロード

In [ ]:
# ダウンロードに失敗しないようにするためのおまじない
os.environ["SSL_CERT_FILE"] = certifi.where()

# モデルのダウンロード先
os.environ["TFHUB_CACHE_DIR"] = Path("~/.cache/tfhub_modules").expanduser().as_posix()

model = hub.load("https://tfhub.dev/tensorflow/ssd_mobilenet_v2/2")

## 画像データをロード

In [ ]:
image = Image.open("dog.jpg")
image

## 画像データをテンソル化

In [ ]:
images = np.asarray(image)
images = tf.expand_dims(images, axis=0)
images.shape, images.dtype

## 推論を実行

In [ ]:
outputs = model(images)

boxes = outputs["detection_boxes"][0]
labels = outputs["detection_classes"][0]
scores = outputs["detection_scores"][0]

for box, label, score in zip(boxes, labels, scores):
    y_min, x_min, y_max, x_max = box
    label = int(label)
    name = category_index[label]["name"]

    if score <= 0.5:
        continue

    print(f"{name},{label},{score:.6f},{x_min:.6f},{y_min:.6f},{x_max:.6f},{y_max:.6f}")

## バウンディングボックスを描画

In [ ]:
output_image = np.asarray(image).copy()

viz_utils.visualize_boxes_and_labels_on_image_array(
    output_image,
    boxes.numpy(),
    labels.numpy().astype(int),
    scores.numpy(),
    category_index,
    use_normalized_coordinates=True,
)

output_image = Image.fromarray(output_image)
output_image